In [ ]:
import pandas as pd
import os

def carregar_descargas(tipo):
    caminho = f'C:\\Users\\rcvs3\\OneDrive - Energisa\\Documentos\\Mestrado\\DESCARGAS_2023\\Consolidado Raios 2023_{tipo}.csv'
    return pd.read_csv(caminho, delimiter=';')

tipos_descargas = ['EPB', 'ERO', 'EMT', 'EMS', 'EMR', 'EAC', 'ETO', 'ESS', 'ESE']
descargas = {tipo: carregar_descargas(tipo) for tipo in tipos_descargas}

# Aplicar operações a cada DataFrame
for tipo, df in descargas.items():
    # Mantendo apenas as colunas da base que possuem dados de raios no sentido Nuvem-Solo
    df = df.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]
    
    # Excluir a primeira linha, pois a tabela só começa na terceira linha
    df = df.drop(df.index[0])
    
    # Renomear todas as colunas para que fique de acordo com o número de raios nos meses correspondentes
    novos_nomes_colunas = ['Nome_Município', 'Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
    df.columns = novos_nomes_colunas
    
    # Atualizar o DataFrame original no dicionário
    descargas[tipo] = df


In [ ]:
Descargas_EPB = descargas['EPB']
Descargas_ERO = descargas['ERO']
Descargas_ETO = descargas['ETO']
Descargas_EMS = descargas['EMS']
Descargas_EMR = descargas['EMR']
Descargas_EMT = descargas['EMT']
Descargas_EAC = descargas['EAC']
Descargas_ESS = descargas['ESS']
Descargas_ESE = descargas['ESE']



In [ ]:
Cod_IBGE = pd.read_excel('C:\\Users\\rcvs3\\OneDrive - Energisa\\Documentos\\Mestrado\\cod_ibge.xls', header=6)

In [ ]:
# Definindo os DataFrames IBGE para cada estado
IBGE_EPB = Cod_IBGE[Cod_IBGE['Nome_UF'] == 'Paraíba']
IBGE_ESE = Cod_IBGE[Cod_IBGE['Nome_UF'] == 'Sergipe']
IBGE_ETO = Cod_IBGE[Cod_IBGE['Nome_UF'] == 'Tocantins']
IBGE_EAC = Cod_IBGE[Cod_IBGE['Nome_UF'] == 'Acre']
IBGE_ERO = Cod_IBGE[Cod_IBGE['Nome_UF'] == 'Rondônia']
IBGE_EMT = Cod_IBGE[Cod_IBGE['Nome_UF'] == 'Mato Grosso']
IBGE_EMS = Cod_IBGE[Cod_IBGE['Nome_UF'] == 'Mato Grosso do Sul']
IBGE_ESS = Cod_IBGE[Cod_IBGE['Nome_UF'].isin(['São Paulo', 'Paraná', 'Minas Gerais'])]
IBGE_EMR = Cod_IBGE[Cod_IBGE['Nome_UF'].isin(['Minas Gerais', 'Rio de Janeiro'])]

# Mesclar cada Descargas_EXX com seu equivalente IBGE_EXX
Descargas_EPB = pd.merge(Descargas_EPB, IBGE_EPB, on='Nome_Município', how='inner')
Descargas_ESE = pd.merge(Descargas_ESE, IBGE_ESE, on='Nome_Município', how='inner')
Descargas_ETO = pd.merge(Descargas_ETO, IBGE_ETO, on='Nome_Município', how='inner')
Descargas_EAC = pd.merge(Descargas_EAC, IBGE_EAC, on='Nome_Município', how='inner')
Descargas_ERO = pd.merge(Descargas_ERO, IBGE_ERO, on='Nome_Município', how='inner')
Descargas_EMT = pd.merge(Descargas_EMT, IBGE_EMT, on='Nome_Município', how='inner')
Descargas_EMS = pd.merge(Descargas_EMS, IBGE_EMS, on='Nome_Município', how='inner')
Descargas_ESS = pd.merge(Descargas_ESS, IBGE_ESS, on='Nome_Município', how='inner')
Descargas_EMR = pd.merge(Descargas_EMR, IBGE_EMR, on='Nome_Município', how='inner')

# Deixando um único dataframe pra todas as descargas do grupo Energisa
Lista_Descargas = [Descargas_EPB, Descargas_ERO, Descargas_EMT, Descargas_EMS, Descargas_EMR, Descargas_EAC, Descargas_ETO, Descargas_ESS, Descargas_ESE]
Descargas_Energisa = pd.concat(Lista_Descargas, ignore_index=True)

In [ ]:
# ADICIONAR COLUNA DE SOMA DOS MESES DE JANEIRO A DEZEMBRO, NO DATAFRAME INTEIRO MESMO.
# Convertendo as colunas de meses para o tipo numérico, substituindo valores não numéricos por 0
Descargas_Energisa.iloc[:, 1:13] = Descargas_Energisa.iloc[:, 1:13].applymap(lambda x: pd.to_numeric(x, errors='coerce')).fillna(0)

# Agora, criamos a coluna "Total" somando os valores das colunas de meses
Descargas_Energisa['Total'] = Descargas_Energisa.iloc[:, 1:13].sum(axis=1)

print(Descargas_Energisa)

print(Descargas_Energisa.columns)

In [ ]:
Ocorrencias = pd.read_csv('C:\\Users\\rcvs3\\OneDrive - Energisa\\Documentos\\Mestrado\\ocorrencias-emergenciais-rede-distribuicao-2023.csv', delimiter=';', encoding='latin-1', low_memory=False)

In [ ]:

# renomeia a coluna CodIBGE para Código Município Completo

Ocorrencias = Ocorrencias.rename(columns={'CodIBGE': 'Código Município Completo'})


# Ocorrencias = Ocorrencias.rename('CodIBGE', 'Código Município Completo')

In [ ]:
print(Ocorrencias.columns)


In [ ]:
print(Descargas_Energisa.columns)

In [ ]:
# Filtrando apenas ocorrências que sejam advindas de descargas atmosféricas
Ocorrencias = Ocorrencias[Ocorrencias['DscOcorrenciaAberta'] == "INTERNA;NAO PROGRAMADA;MEIO AMBIENTE;DESCARGA ATMOSFERICA"]
# # Mesclando a tabela de Ocorrências com o nome dos municípios, conforme código IBGE
Ocorrencias = pd.merge(Ocorrencias, Cod_IBGE, on='Código Município Completo', how='inner')
Ocorrencias = pd.merge(Ocorrencias, Descargas_Energisa, on='Código Município Completo', how='inner')

In [ ]:
print(Ocorrencias)

In [ ]:
print(Ocorrencias.columns)


In [ ]:
# Criando a coluna que concatena 'Nome_Município_x' e 'Nome_UF_x'
Ocorrencias['Município - Estado'] = Ocorrencias.apply(lambda row: f"{row['Nome_Município_x']} - {row['Nome_UF_x']}", axis=1)

# Agrupando e contando as ocorrências
df = Ocorrencias.groupby(['Nome_Município_x', 'Nome_UF_x', 'Município - Estado', 'Total', 'Código Município Completo']).size().reset_index(name='Contagem de Ocorrências')

df = pd.merge(df, Descargas_Energisa, on='Código Município Completo', how='inner')

print(df)

# Criar tabela com todas as informações, que vão ser jogadas lá no yugabyte
